# Data Formats and Persistence

## Pickle

**`pickle`** este un modul standard în Python care permite **serializarea** și **deserializarea** obiectelor. Prin „serializare” (numită și *pickling*), un obiect Python este transformat într-un flux de **bytes**, care poate fi:
- stocat într-un fișier de tip `.pckl` / `.pkl`;

- transmis prin rețea;

- salvat temporar în memorie pentru utilizare ulterioară.

Procesul invers, *deserializarea* (sau *unpickling*), reconstruiește obiectul original din fluxul de bytes. Acest mecanism este esențial pentru salvarea stării aplicațiilor, comunicarea între procese sau persistarea datelor complexe.

Funcțiile principale ale modulului `pickle` sunt:
- `pickle.dump(obj, file, protocol=None)` — Scrie obiectul serializat într-un fișier deschis în mod binar (`'wb'`). Creează o reprezentare binară a obiectului și o salvează pe disc.
    - `obj` — obiectul Python care urmează să fie serializat.  
    - `file` — obiect fișier deschis pentru scriere binară.  
    - `protocol` — versiunea protocolului de serializare (implicit cea mai nouă).  

- `pickle.load(file)` — Reconstruiește obiectul original dintr-un fișier deschis în mod binar (`'rb'`). Returnează o copie completă a obiectului inițial.   
    - `file` — fișierul care conține datele serializate.  

- `pickle.dumps(obj, protocol=None)` — Serializarea unui obiect în **memorie**, returnând un flux de tip `bytes`. Utilă pentru trimiterea obiectelor prin rețea sau stocarea lor în baze de date. 
    - `obj` — obiectul de serializat.  
    - `protocol` — opțional, stabilește versiunea de serializare.  

- `pickle.loads(data)` — Reconstruiește obiectul original dintr-un flux de **bytes**. Returnează obiectul Python reconstituit în memorie, identic logic cu cel original.
    - `data` — datele serializate (tip `bytes`).  

Modulul `pickle` poate serializa nu doar tipuri primitive (liste, dicționare, șiruri, numere), ci și **instanțe de clase definite de utilizator**. Atunci când o instanță este serializată, starea internă a obiectului (atributele sale) este salvată, iar la deserializare, obiectul este reconstruit și poate fi folosit ca înainte. Pentru ca un obiect să fie serializabil, clasa sa trebuie să fie definită la nivelul superior al unui modul (nu în interiorul unei funcții) și să nu includă resurse neserializabile.

Un fișier `.pckl` / `.pkl` poate fi deschis ca:
- `'wb'` — **write binary**, pentru a scrie date serializate într-un fișier nou.  

- `'rb'` — **read binary**, pentru a citi și reconstrui obiectul serializat.  

- `'ab'` — **append binary**, pentru a adăuga date serializate la finalul unui fișier existent.  

- `'xb'` — **exclusive binary**, pentru a crea un fișier nou și a eșua dacă acesta există deja.

Obiectele care nu pot fi serializate în `pickle` sunt:
- Obiecte care conțin **referințe către funcții lambda**, **generatoare** sau **clase locale**

- Obiecte externe (ex: conexiuni la baze de date, handle-uri GPU, etc.)

- Obiecte care reprezintă **resurse de sistem**, cum ar fi:
  - Socket-uri deschise (`socket.socket`)
  - Thread-uri (`threading.Thread`)
  - Obiecte de tip **lock** sau **semaphore**
  - Fișiere deschise (`open()`)

In [ ]:
import pickle

car = {"brand": "Ford", "model": "Mustang", "year": 1965}

with open("data.pckl", "wb") as f:
    pickle.dump(car, f)
print("[1] Dumped car to file:", car)

with open("data.pckl", "rb") as f:
    car_loaded = pickle.load(f)
print("[1] Loaded car from file:", car_loaded, '\n')

data_bytes = pickle.dumps(car)
print("[2] Type after dumps:", type(data_bytes).__name__)

car_from_memory = pickle.loads(data_bytes)
print("[2] Restored from memory:", car_from_memory, '\n')

class Employee:
    def __init__(self, name, position, salary):
        self.name = name
        self.position = position
        self.salary = salary

    def promote(self, new_position, raise_amount):
        self.position = new_position
        self.salary += raise_amount

    def __repr__(self):
        return f"Employee({self.name}, {self.position}, {self.salary})"


team = [
    Employee("Alice", "Developer", 70000),
    Employee("Bob", "Designer", 65000),
    Employee("Charlie", "Manager", 85000),
]

bundle = {"team": team, "department": "R&D", "year": 2025}

with open("employees.pkl", "wb") as f:
    pickle.dump(bundle, f)

with open("employees.pkl", "rb") as f:
    bundle_loaded = pickle.load(f)

print("[3] Loaded bundle keys:", list(bundle_loaded.keys()))
print("[3] Type of first employee:", type(bundle_loaded["team"][0]).__name__)
print("[3] Example restored object:", bundle_loaded["team"][0])

[1] Dumped car to file: {'brand': 'Ford', 'model': 'Mustang', 'year': 1965}
[1] Loaded car from file: {'brand': 'Ford', 'model': 'Mustang', 'year': 1965} 

[2] Type after dumps: bytes
[2] Restored from memory: {'brand': 'Ford', 'model': 'Mustang', 'year': 1965} 

[3] Loaded bundle keys: ['team', 'department', 'year']
[3] Type of first employee: Employee
[3] Example restored object: Employee(Alice, Developer, 70000)


## Shelve

**`shelve`** este un modul standard în Python care oferă o metodă simplă de **stocare persistentă a obiectelor**. Acesta combină funcționalitatea modulelor `pickle` și `dbm`, permițând stocarea obiectelor Python într-o **bază de date pe disc** care se comportă similar cu un **dicționar persistent**. Cheile dintr-un fișier shelve trebuie să fie **șiruri de caractere (string)**, iar valorile pot fi **orice obiecte serializabile** cu `pickle`. Accesul la date se face la fel ca în cazul unui dicționar Python obișnuit.

Funcțiile și metodele principale ale modulului shelve sunt:
- `shelve.open(filename, flag='c', protocol=None, writeback=False)` — Deschide o bază de date `shelve` și returnează un obiect asemănător unui dicționar.  
    - `filename` — numele fișierului (fără extensie), modulul creează mai multe fișiere interne pentru stocare (`.dat`, `.dir`, `.bak`).  
    - `flag` — definește modul de acces:  
        - `'c'` — **create**: creează fișierul dacă nu există (implicit).  
        - `'n'` — **new**: creează întotdeauna o bază de date nouă, suprascriind fișierele existente.  
        - `'r'` — **read-only**: deschide baza de date doar pentru citire.  
        - `'w'` — **write**: deschide baza de date existentă pentru scriere și citire (eroare dacă nu există).  
    - `protocol` — stabilește protocolul de serializare `pickle` (implicit, cel mai recent).  
    - `writeback` — dacă este `True`, valorile modificate în memorie sunt scrise automat înapoi în fișier la închiderea bazei de date (consumă mai multă memorie).  

Obiectele care **nu pot fi stocate** într-o bază de date `shelve` sunt aceleași care nu pot fi serializate prin `pickle`, cum ar fi:
- Obiecte care conțin **referințe către funcții lambda**, **generatoare** sau **clase locale**

- Obiecte externe (ex: conexiuni la baze de date, handle-uri GPU, etc.)

- Obiecte care reprezintă **resurse de sistem**, cum ar fi:
  - Socket-uri deschise (`socket.socket`)
  - Thread-uri (`threading.Thread`)
  - Obiecte de tip **lock** sau **semaphore**
  - Fișiere deschise (`open()`)

In [3]:
import shelve

class Employee:
    def __init__(self, name, role, salary):
        self.name = name
        self.role = role
        self.salary = salary

    def promote(self, new_role, raise_amount):
        self.role = new_role
        self.salary += raise_amount

    def __repr__(self):
        return f"Employee({self.name!r}, {self.role!r}, {self.salary!r})"


def create_shelf(path= "company"):
    with shelve.open(path, flag="n") as db:
        db["name"] = "Apple"
        db["country"] = "USA"
        db["currency"] = ["USD", "$"]
        db["meta"] = {"founded": 1976, "public": True}

        db["employees"] = [
            Employee("Alice", "Developer", 70000),
            Employee("Bob", "Designer", 65000),
            Employee("Charlie", "Manager", 85000),
        ]

        print("[CREATE] Wrote initial entries:", list(db.keys()), '\n')

def inspect_shelf(path= "company"):
    with shelve.open(path, flag='c') as db:
        print("[INSPECT] keys ->", list(db.keys()))
        print("[INSPECT] name ->", db["name"])
        print("[INSPECT] currency ->", db["currency"])
        print("[INSPECT] country exists? ->", "country" in db)
        print("[INSPECT] get('missing', 'N/A') ->", db.get("missing", "N/A"))
        print("[INSPECT] meta ->", db["meta"])

        print("[INSPECT] employees ->")
        for e in db["employees"]:
            print("   ", e)
        print()

def update_with_write_flag(path= "company"):
    with shelve.open(path, flag="w") as db:
        db["currency"] = ["USD", "$", "cents"]
        print("[UPDATE] currency updated to:", db["currency"])

        emps = db["employees"]
        emps[0].promote("Senior Developer", 5000)
        db["employees"] = emps
        print("[UPDATE] promoted:", emps[0], '\n')

create_shelf("company")
inspect_shelf("company")
update_with_write_flag("company")
inspect_shelf("company")

[CREATE] Wrote initial entries: ['country', 'currency', 'employees', 'meta', 'name'] 

[INSPECT] keys -> ['country', 'currency', 'employees', 'meta', 'name']
[INSPECT] name -> Apple
[INSPECT] currency -> ['USD', '$']
[INSPECT] country exists? -> True
[INSPECT] get('missing', 'N/A') -> N/A
[INSPECT] meta -> {'founded': 1976, 'public': True}
[INSPECT] employees ->
    Employee('Alice', 'Developer', 70000)
    Employee('Bob', 'Designer', 65000)
    Employee('Charlie', 'Manager', 85000)

[UPDATE] currency updated to: ['USD', '$', 'cents']
[UPDATE] promoted: Employee('Alice', 'Senior Developer', 75000) 

[INSPECT] keys -> ['country', 'currency', 'employees', 'meta', 'name']
[INSPECT] name -> Apple
[INSPECT] currency -> ['USD', '$', 'cents']
[INSPECT] country exists? -> True
[INSPECT] get('missing', 'N/A') -> N/A
[INSPECT] meta -> {'founded': 1976, 'public': True}
[INSPECT] employees ->
    Employee('Alice', 'Senior Developer', 75000)
    Employee('Bob', 'Designer', 65000)
    Employee('Char

## JSON

**`json`** este un modul standard în Python care oferă funcționalități pentru **serializarea** și **deserializarea** datelor în formatul **JSON (JavaScript Object Notation)**. Este unul dintre cele mai folosite formate de schimb de date între aplicații, datorită faptului că este **ușor de citit de oameni** și **independent de limbaj**. Formatul JSON reprezintă o structură bazată pe perechi cheie–valoare, similară cu un dicționar Python, și poate fi utilizată pentru a transfera obiecte complexe între programe sau prin rețea.

Un fișier JSON conține diferite tipuri de date, iar conversia la tipurile de date din Python are loc in mod automat la **serializare** și **deserializare**, după cum urmează:
| JSON                 | Python           |
|--------|------|
| Obiecte `{}`         | `dict`           |
| Liste `[]`           | `list` / `tuple` |
| `string`             | `str`            |
| `number`             | `int` / `float`  |
| `true` / `false`     | `True` / `False` |
| `null`               | `None`           |

Funcțiile principale ale modulului `json` sunt:
- `with open(filename, mode='r') as file:` — Un fișier JSON este un **fișier text**, prin urmare, înainte de utilizarea metodelor `dump()` pentru **serializare** sau `load()` pentru **deserializare**, fișierul trebuie deschis corespunzător.
    - `filename` — numele fișierului. 
    - `mode` — definește modul de acces:    
        - `'r'` — **read**: deschide fișierul pentru citire, utilizat împreună cu funcțiile `load()` pentru a **deserializa** conținutul JSON într-un obiect Python.
        - `'w'` — **write**: deschide fișierul pentru scriere, suprascriind conținutul existent, folosit cu `dump()` pentru a **scrie datele serializate** în format JSON pe disc.
        - `'a'` — **append**: deschide fișierul pentru adăugare la finalul conținutului existent.  
        - `'x'` — **exclusive creation**: creează un fișier nou și generează o eroare (`FileExistsError`) dacă fișierul există deja.

- `json.load(file, *, cls=None)` — Citește un fișier JSON și reconstruiește obiectul Python corespunzător.  
    - `file` — fișier deschis pentru citire.  
    - `cls` — opțional, o clasă `JSONDecoder` personalizată pentru conversia în tipuri de obiecte definite de utilizator.  

- `json.loads(s, *, cls=None)` — Deserializarea unui șir JSON (`str`) în memorie.  
    - `s` — șirul JSON de interpretat (`str`).  
    - `cls` — opțional, o clasă `JSONDecoder` personalizată pentru conversia în tipuri de obiecte definite de utilizator.  

- `json.dump(obj, file, *, cls=None, indent=None)` — Scrie obiectul serializat în format JSON într-un fișier text.  
    - `obj` — obiectul Python care urmează să fie serializat.  
    - `file` — obiect fișier deschis pentru scriere.  
    - `cls` — opțional, o clasă `JSONEncoder` personalizată pentru serializarea obiectelor complexe.  
    - `indent` — număr de spații pentru formatarea frumoasă (*pretty-printing*).  

- `json.dumps(obj, *, cls=None, indent=None)` — Serializarea unui obiect Python în **memorie**, returnând un șir JSON (`str`).  
    - `obj` — obiectul de convertit.  
    - `cls` — opțional, o clasă `JSONEncoder` personalizată pentru serializarea obiectelor complexe.  
    - `indent` — număr de spații pentru formatarea frumoasă (*pretty-printing*). 

Pentru obiecte complexe (ex: instanțe de clase definite de utilizator), **JSON** nu știe nativ cum să le serializeze. Pentru a gestiona acest lucru, se pot defini **clase personalizate** pentru encoder și decoder.

Un **encoder personalizat** este o clasă implementată de utilizator care moștenește `json.JSONEncoder`. Această clasă trebuie să implementeze:
- `default(self, obj)` - Metodă care transformă obiectul neserializabil într-unul serializabil.
    - `obj` - obiectul necunoscut care se dorește a fi serializat.
    - se returnează o versiune serializabilă a obiectului (de exemplu, un dicționar), iar dacă tipul nu este recunoscut, se apelează metoda din clasa părinte.

Un **decoder personalizat** este o clasă implementată de utilizator care moștenește `json.JSONDecoder`. Această clasă trebuie să implementeze:
- `__init__(self, *args, **kwargs)` - În cadrul inițializării, trebuie transmisă o funcție `object_hook` ca parametru către superclasă.

- `object_hook(self, obj)` - Metoda care transformă obiectele decodate din JSON în instanțe de clase personalizate. Reconstruiește obiectul original dacă este cazul, în caz contrar, returnează obiectul neschimbat.
    - `obj` - obiect JSON decodat (sub formă de dicționar).

Obiectele care nu pot fi serializate în JSON:
- Instanțe ale claselor Python (fără un `Encoder` personalizat).  

- Obiecte care conțin **funcții**, **metode**, **generatoare** sau **lambda-uri**.  

- Obiecte care conțin **referințe circulare** (ex: o listă care se conține pe ea însăși).

- Obiecte externe (ex: conexiuni la baze de date, handle-uri GPU etc.)

- Obiecte care reprezintă **resurse de sistem**, cum ar fi:
  - Socket-uri deschise (`socket.socket`)
  - Thread-uri (`threading.Thread`)
  - Obiecte de tip **lock** sau **semaphore**
  - Fișiere deschise (`open()`)


Erorile și excepțiile din modulul JSON sunt:
- `json.JSONDecodeError` — apare dacă șirul JSON este invalid sintactic (caractere lipsă, acolade incorecte etc.).  

- `TypeError` — apare când se încearcă serializarea unui obiect nesuportat fără encoder personalizat.  

- `ValueError` — poate apărea la conversii numerice invalide sau la încărcarea datelor corupte.  

- `UnicodeEncodeError` / `UnicodeDecodeError` — erori la procesarea textului cu caractere speciale.  

In [ ]:
import json
from pathlib import Path

class Employee:
    def __init__(self, name, position, salary):
        self.name = name
        self.position = position
        self.salary = float(salary)

    def promote(self, new_position, raise_amount):
        self.position = new_position
        self.salary += float(raise_amount)

    def __repr__(self):
        return f"Employee({self.name!r}, {self.position!r}, {self.salary!r})"

class EmployeeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Employee):
            return {
                "__type__": "Employee",
                "name": obj.name,
                "position": obj.position,
                "salary": obj.salary,
            }
        return super().default(obj)


class EmployeeDecoder(json.JSONDecoder):
    def __init__(self, *args, **kwargs):
        super().__init__(object_hook=self._object_hook, *args, **kwargs)

    @staticmethod
    def _object_hook(obj):
        if obj.get("__type__") == "Employee":
            return Employee(
                name=obj["name"],
                position=obj["position"],
                salary=obj["salary"],
            )
        return obj


json_input = """
{
  "team": [
    {"__type__": "Employee", "name": "Alice", "position": "Developer", "salary": 70000},
    {"__type__": "Employee", "name": "Bob", "position": "Designer", "salary": 65000},
    {"__type__": "Employee", "name": "Charlie", "position": "Manager", "salary": 85000}
  ],
  "department": "R&D",
  "year": 2025
}
"""

bundle = json.loads(json_input, cls=EmployeeDecoder)
print("[LOADS] type(bundle):", type(bundle).__name__)
print("[LOADS] type(bundle['team'][0]):", type(bundle["team"][0]).__name__)
print("[LOADS] exemple Employee object:", bundle["team"][0], '\n')

bundle["team"][0].promote("Senior Developer", 5000)

json_path = Path("employees.json")
with json_path.open("w") as f:
    json.dump(bundle, f, cls=EmployeeEncoder, indent=2)
print(f"[DUMP] wrote JSON to: {json_path.resolve()}\n")

with json_path.open("r") as f:
    bundle_loaded = json.load(f, cls=EmployeeDecoder)
print("[LOAD] type(bundle_loaded):", type(bundle_loaded).__name__)
print("[LOAD] restored Employee sample:", bundle_loaded["team"][0], '\n')

json_output = json.dumps(bundle_loaded, cls=EmployeeEncoder, indent=2)
print("[DUMPS] type(json_output):", type(json_output).__name__)
print("[DUMPS] JSON string length:", len(json_output))
print("[DUMPS] preview:")
print(json_output)

[LOADS] type(bundle): dict
[LOADS] type(bundle['team'][0]): Employee
[LOADS] exemple Employee object: Employee('Alice', 'Developer', 70000.0) 

[DUMP] wrote JSON to: C:\Endava\EndevLocal\pcpp\employees.json

[LOAD] type(bundle_loaded): dict
[LOAD] restored Employee sample: Employee('Alice', 'Senior Developer', 75000.0) 

[DUMPS] type(json_output): str
[DUMPS] JSON string length: 424
[DUMPS] preview:
{
  "team": [
    {
      "__type__": "Employee",
      "name": "Alice",
      "position": "Senior Developer",
      "salary": 75000.0
    },
    {
      "__type__": "Employee",
      "name": "Bob",
      "position": "Designer",
      "salary": 65000.0
    },
    {
      "__type__": "Employee",
      "name": "Charlie",
      "position": "Manager",
      "salary": 85000.0
    }
  ],
  "department": "R&D",
  "year": 2025
}


## DTD

**DTD (Document Type Definition)** este un mecanism folosit pentru a **defini structura, elementele și atributele** unui document XML. Prin DTD se specifică **ce elemente sunt permise**, **cum sunt organizate**, **ce tip de conținut pot avea** și **ce atribute sunt valide** pentru fiecare element. Astfel, un DTD oferă o „schemă" care validează dacă un fișier XML respectă o anumită structură logică.

Un fișier DTD are următoarea structura:
``` dtd
<!ELEMENT root_element (child_element*)>
```
Definește elementul `root_element`, adică elementul rădăcină al documentului XML. Conține unul sau mai multe elemente `child_element`. Simbolul `*` înseamnă „zero sau mai multe apariții".

``` dtd
<!ELEMENT child_element (subelement1, subelement2)>
```
Definește structura elementului `child_element`. Acesta trebuie să conțină exact două subelemente, `subelement1` urmat de
`subelement2`. Ordinea este importantă în DTD: `subelement1` trebuie să apară înaintea lui `subelement2`.

``` dtd
<!ATTLIST child_element attribute CDATA #REQUIRED>
```
Definește o listă de atribute pentru elementul `child_element`. Atributul se numește `attribute` și este de tip `CDATA` (Character Data -- text liber). Marcajul `#REQUIRED` indică faptul că acest atribut este obligatoriu în fiecare apariție a elementului `child_element`.

``` dtd
<!ELEMENT subelement1 (#PCDATA)>
<!ELEMENT subelement2 (#PCDATA)>
```
Specifică faptul că elementele `subelement1` și `subelement2` conțin text simplu (`Parsed Character Data`). `#PCDATA` permite includerea de text care poate fi analizat (parsabil), adică XML-ul poate interpreta entități și caractere speciale din interiorul acestuia.

Următorul document XML respectă exact regulile definite mai sus în DTD:
``` xml
<?xml version="1.0" encoding="UTF-8"?>
<root_element>
    <child_element attribute="example-1">
        <subelement1>This is text inside the first subelement.</subelement1>
        <subelement2>This is text inside the second subelement.</subelement2>
    </child_element>

    <child_element attribute="example-2">
        <subelement1>Another text block here.</subelement1>
        <subelement2>And more text here.</subelement2>
    </child_element>
</root_element>
```

## XML

**XML (eXtensible Markup Language)** este un limbaj de marcare folosit pentru **stocarea și schimbul de date** într-un format ierarhic, lizibil de oameni și de mașini. Spre deosebire de JSON, XML oferă o structură clară bazată pe **etichete (tag-uri)** și **atribute**, permițând descrierea datelor și a relațiilor dintre ele.

Un fișier XML este sub forma: 
```xml
<?xml version="1.0" encoding="UTF-8"?>
<tag attribute="value">text</tag>
```
- `version` - Specifică versiune.

- `encoding` - Specifică setul de caractere.

- `tag` - Reprezintă unități logice de date. Fiecare element are o **etichetă de deschidere** și una de **închidere**.

- `attribute` - Adaugă informații suplimentare elementelor, în interiorul etichetei de deschidere.

- `text` - Este conținutul efectiv al elementului, aflat între etichete.

Modulul `xml.etree.ElementTree` face parte din biblioteca standard Python și oferă instrumente pentru crearea, parcurgerea, modificarea și scrierea fișierelor XML. El reprezintă documentele XML sub formă de structuri ierarhice de obiecte, unde fiecare nod este un element XML.

Funcțiile și clasele principale ale modului sunt:
- `Element(tag, attrib={}, **extra)` - Creează un element XML.
    - `tag` - numele etichetei elementului.
    - `attrib` - dicționar cu atributele elementului.
    - `**extra` - parametri suplimentari pentru atribute.
    - returnează un obiect `Element` care poate conține subelemente, text și atribute.

- `SubElement(parent, tag, attrib={}, **extra)` - Creează un subelement în interiorul unui element existent.
    - `parent` - elementul părinte în care se inserează subelementul.
    - `tag` - numele subelementului.
    - `attrib` - dicționar de atribute.
    - `**extra` - parametri suplimentari pentru atribute.
    - returnează un obiect `Element` care devine copilul elementului `parent`.

- `ElementTree(element=None)` - Creează o structură de tip arbore care conține un element rădăcină.
    - `element` - elementul rădăcină (root) al arborelui XML.
    - returnează un obiect `ElementTree` care poate fi salvat în fișier.

- `write(file, encoding="utf-8", xml_declaration=True, method="xml", **kwargs)` - Scrie arborele XML într-un fișier.
    - `file` - numele fișierului sau obiectul de fișier.
    - `encoding` - setul de caractere utilizat (implicit `utf-8`).
    - `xml_declaration` - dacă este `True`, include linia `<?xml ... ?>`.
    - `method` - formatul de scriere (`xml`, `html`, `text`).

- `parse(source)` - Citește un fișier XML și creează un obiect `ElementTree`.
    - `source` - calea către fișierul XML.
    - returnează un obiect `ElementTree`.

- `getroot()` - Obține elementul rădăcină al unui arbore XML.
    - returnează un obiect `Element` corespunzător rădăcinii.

- `findall(path)` - Caută și returnează toate elementele care corespund unei căi XPath simple.
    - `path` - expresia XPath (ex: `"book"` sau `"book/author"`).
    - returnează o listă de obiecte `Element` sau `None` dacă nu este găsit.

- `find(path)` - Returnează primul element care corespunde unei căi XPath.
    - `path` - expresia XPath.
    - returnează un obiect `Element` sau `None` dacă nu este găsit.

- `set(key, value)` - Adaugă sau modifică un atribut al elementului.
    - `key` - numele atributului.
    - `value` - valoarea atributului.

- `get(key, default=None)` - Returnează valoarea unui atribut al elementului.
    - `key` - numele atributului.
    - `default` - valoarea returnată dacă atributul nu există.
    - returnează valoarea atributului sau `default`.

- `tostring(element, encoding="utf-8", method="xml")` - Convertește un element XML (și subelementele sale) într-un șir de bytes.
    - `element` - elementul XML de convertit.
    - `encoding` - tipul de codare (`utf-8`, `unicode`).
    - `method` - formatul de ieșire (`xml`, `html`, `text`).
    - returnează un obiect `bytes` ce conține conținutul XML serializat.

- `fromstring(text)` - Creează o structură XML în memorie pe baza unui șir de text.
    - `text` - conținutul XML în format string.
    - returnează un obiect `Element` reprezentând elementul rădăcină.

- `dump(element)` - Afișează structura XML în consolă într-un format lizibil.
    - `element` - elementul care trebuie afișat.

- `xpath(path, namespaces=None, smart_strings=True, **variables)` – Evaluează o expresie XPath relativ la un element sau la întregul document și returnează rezultatele selecției.
    - `path` – expresia XPath care trebuie evaluată.
    - `namespaces` – dicționar de mapări prefix → URI pentru documentele care folosesc spații de nume (de exemplu: `{"x": "http://example.com/ns"}`).
    - `smart_strings` – dacă este `True`, valorile text/atribute returnate păstrează legătura cu nodul sursă (comportament specific implementărilor tip lxml), dar dacă este `False`, se întorc șiruri simple.
    - `**variables` – variabile XPath injectate în expresie (pot fi folosite în condiții).
    - `returnează, în funcție de expresie:
        - o listă de noduri `Element` când expresia selectează elemente (nodeset);
        - o listă de șiruri pentru selecții de text sau atribute (de exemplu: `text()`, `@attr`);
        - o valoare primitivă (`str`, `float`, `bool`) dacă expresia produce direct un tip scalar (de exemplu: `string(.)`, `number(.)`, `boolean(.)`).

## XPath

**XPath (XML Path Language)** este un limbaj folosit pentru a naviga și a selecta noduri din documente XML. Acesta permite identificarea precisă a elementelor și a atributelor dintr-o structură ierarhică, oferind un mod flexibil de a extrage datele relevante dintr-un document marcat.

XPath permite exprimarea unor condiții complexe asupra elementelor, atributelor și valorilor lor. Printre cele mai frecvente tipuri de filtrări și condiții se numără:
```xml
<?xml version="1.0" encoding="UTF-8"?>
<root_element>
    <meta>
        <author>Author</author>
        <tags>
            <tag priority="1">demo</tag>
            <tag>example</tag>
        </tags>
    </meta>

    <child_element attribute="example-1" id="c1" status="active">
        <subelement1 lang="en">This is text inside the first subelement.</subelement1>
        <subelement2>This is text inside the second subelement.</subelement2>

        <details>
            <created>2025-11-01</created>
            <updated>2025-11-24T08:30:00+02:00</updated>
            <flags>
                <flag name="beta" value="true"/>
                <flag name="archived" value="false"/>
            </flags>
        </details>

        <items>
            <item index="1">Alpha</item>
            <item index="2" ref="#c3">Linked</item>
            <item index="3"/>
        </items>
    </child_element>

    <child_element attribute="example-2" id="c2" status="draft">
        <subelement1>Another text block here.</subelement1>
        <subelement2>And more text here.</subelement2>
        <attachments>
            <file name="report.pdf" size="1048576" mime="application/pdf"/>
            <file name="image.png" size="20480" mime="image/png" checksum="sha256:d41d8cd98f00b204e9800998ecf8427e"/>
        </attachments>
    </child_element>

    <child_element attribute="example-3" id="c3" status="inactive">
        <subelement1 xml:space="preserve">Preserved whitespace start and new line</subelement1>
        <subelement2>Value with special chars: &amp;, &lt;, &gt;, &quot;.</subelement2>
        <subelement3 xsi:nil="true"/>
        <relations>
            <rel type="depends-on" target="#c1"/>
            <rel type="duplicates" target="#c2"/>
        </relations>
    </child_element>

    <summary total_children="3">
        <counts>
            <byStatus active="1" draft="1" inactive="1"/>
        </counts>
    </summary>
</root_element>
```

- Toate elementele `child_element` din documentul XML:
  ```xpath
  /root_element/child_element
  ```
  Returnează cele 3 noduri `<child_element>` de sub rădăcină:
  ```xml
  <child_element attribute="example-1" id="c1" status="active">...</child_element>
  <child_element attribute="example-2" id="c2" status="draft">...</child_element>
  <child_element attribute="example-3" id="c3" status="inactive">...</child_element>
  ```

- Toate id-urile elementelor `child_element`:
  ```xpath
  /root_element/child_element/@id
  ```
  Afișează valorile atributelor `id`:
  ```
  c1
  c2
  c3
  ```

- Toate nodurile `subelement1`, indiferent unde se află:
  ```xpath
  //subelement1
  ```
  Returnează toate cele 3 noduri și conținutul lor textual:
  ```xml
  <subelement1 lang="en">This is text inside the first subelement.</subelement1>
  <subelement1>Another text block here.</subelement1>
  <subelement1 xml:space="preserve">Preserved whitespace start and new line</subelement1>
  ```

- Toate elementele `child_element` cu `status="active"`:
  ```xpath
  /root_element/child_element[@status="active"]
  ```
  Returnează un singur nod:
  ```xml
  <child_element attribute="example-1" id="c1" status="active">...</child_element>
  ```

- Toate elementele `child_element` care au cel puțin un copil `attachments/file`:
  ```xpath
  /root_element/child_element[attachments/file]
  ```
  Returnează `child_element` care conține atașamente:
  ```xml
  <child_element attribute="example-2" id="c2" status="draft">...</child_element>
  ```

- Toate `child_element` care nu conțin elementul `attachments`:
  ```xpath
  /root_element/child_element[not(attachments)]
  ```
  Returnează nodurile fără atașamente:
  ```xml
  <child_element attribute="example-1" id="c1" status="active">...</child_element>
  <child_element attribute="example-3" id="c3" status="inactive">...</child_element>
  ```

- Toate `item` din orice `items` care au atribut `index` > 1:
  ```xpath
  //items/item[@index > 1]
  ```
  Afișează elementele cu index mai mare ca 1:
  ```xml
  <item index="2" ref="#c3">Linked</item>
  <item index="3"/>
  ```

- Toate fișierele PNG:
  ```xpath
  //attachments/file[@mime="image/png"]
  ```
  Returnează fișierul PNG:
  ```xml
  <file name="image.png" size="20480" mime="image/png" checksum="sha256:d41d8cd98f00b204e9800998ecf8427e"/>
  ```

- Toate tag-urile care au atributul `priority`:
  ```xpath
  /root_element/meta/tags/tag[@priority]
  ```
  Afișează tag-ul cu prioritate setată:
  ```xml
  <tag priority="1">demo</tag>
  ```

- Primul `child_element` din listă:
  ```xpath
  (/root_element/child_element)[1]
  ```
  Returnează:
  ```xml
  <child_element attribute="example-1" id="c1" status="active">...</child_element>
  ```

- Ultimul `item` din orice listă `items`:
  ```xpath
  //items/item[last()]
  ```
  Afișează ultimul element din `items` (al lui `c1`):
  ```xml
  <item index="3"/>
  ```

- Toți descendenții `subelement2` ai elementului activ:
  ```xpath
  /root_element/child_element[@status="active"]//subelement2
  ```
  Afișează nodul `subelement2` din `c1`:
  ```xml
  <subelement2>This is text inside the second subelement.</subelement2>
  ```

- Strămoșii unui fișier anume:
  ```xpath
  //file[@name="image.png"]/ancestor::child_element
  ```
  Returnează părintele acelui fișier:
  ```xml
  <child_element attribute="example-2" id="c2" status="draft">...</child_element>
  ```

- Toți frații următori ai oricărui `subelement1`:
  ```xpath
  //subelement1/following-sibling::*
  ```
  Returnează, pentru fiecare `subelement1`, nodurile de pe același nivel care îl urmează:
  ```xml
  <!-- în c1: -->
  <subelement2>...</subelement2>
  <details>...</details>
  <items>...</items>
  <!-- în c2: -->
  <subelement2>...</subelement2>
  <attachments>...</attachments>
  <!-- în c3: -->
  <subelement2>...</subelement2>
  <subelement3 xsi:nil="true"/>
  <relations>...</relations>
  ```

- Părintele unui `rel` cu `type="duplicates"`:
  ```xpath
  //rel[@type="duplicates"]/parent::relations
  ```
  Returnează elementul container:
  ```xml
  <relations>
      <rel type="depends-on" target="#c1"/>
      <rel type="duplicates" target="#c2"/>
  </relations>
  ```

- Toate `subelement2` care conțin simbolul `&` în text:
  ```xpath
  //subelement2[contains(., "&")]
  ```
  Returnează doar nodul din `c3`:
  ```xml
  <subelement2>Value with special chars: &amp;, &lt;, &gt;, &quot;.</subelement2>
  ```

- Toate `child_element` cu atribut `attribute` care începe cu `example-`:
  ```xpath
  /root_element/child_element[starts-with(@attribute, "example-")]
  ```
  Returnează toate cele trei noduri `child_element`
  ```xml
  <child_element attribute="example-1" id="c1" status="active">...</child_element>
  <child_element attribute="example-2" id="c2" status="draft">...</child_element>
  <child_element attribute="example-3" id="c3" status="inactive">...</child_element>
  ```

- Toate `subelement1` al căror text nu este gol:
  ```xpath
  //subelement1[normalize-space(.) != ""]
  ```
  Returnează toate cele trei noduri `subelement1`:
  ```xml
  <subelement1 lang="en">This is text inside the first subelement.</subelement1>
  <subelement1>Another text block here.</subelement1>
  <subelement1 xml:space="preserve">Preserved whitespace start and new line</subelement1>
  ```

- Toate atributele `name` din elementele `flag`:
  ```xpath
  //flag/@name
  ```
  Afișează valorile:
  ```
  beta
  archived
  ```

- Toate fișierele care au atributul `checksum`:
  ```xpath
  //attachments/file[@checksum]
  ```
  Returnează:
  ```xml
  <file name="image.png" size="20480" mime="image/png" checksum="sha256:d41d8cd98f00b204e9800998ecf8427e"/>
  ```

- Toate `subelement1` și `subelement2` (uniune):
  ```xpath
  //subelement1 | //subelement2
  ```
  Afișează toate aceste noduri (6 în total): `subelement1`×3 și `subelement2`×3.

- Toate `child_element` cu `status` „active” sau „draft”:
  ```xpath
  /root_element/child_element[@status="active" or @status="draft"]
  ```
  Returnează:
  ```xml
  <child_element attribute="example-1" id="c1" status="active">...</child_element>
  <child_element attribute="example-2" id="c2" status="draft">...</child_element>
  ```

- Toate `tag` fără atribut `priority`:
  ```xpath
  /root_element/meta/tags/tag[not(@priority)]
  ```
  Afișează:
  ```xml
  <tag>example</tag>
  ```

- `item` care au `@ref="#c3"`:
  ```xpath
  //items/item[@ref = "#c3"]
  ```
  Returnează:
  ```xml
  <item index="2" ref="#c3">Linked</item>
  ```

- Toate fișierele cu dimensiune mai mare de 1 MB:
  ```xpath
  //attachments/file[number(@size) > 1048576]
  ```
  Nu returnează niciun nod (0 rezultate), deoarece avem dimensiuni `1048576` (=1MB) și `20480` (<1MB).

- Numărul total de `child_element`:
  ```xpath
  count(/root_element/child_element)
  ```
  Afișează `3`.

- Numărul `child_element` cu status diferit de `inactive`:
  ```xpath
  count(/root_element/child_element[@status != "inactive"])
  ```
  Afișează `2` (pentru `c1` și `c2`).

- Toate fișierele al căror nume se termină cu `.png` (XPath 2.0):
  ```xpath
  //file[matches(@name, "\.png$")]
  ```
  Returnează:
  ```xml
  <file name="image.png" size="20480" mime="image/png" checksum="sha256:d41d8cd98f00b204e9800998ecf8427e"/>
  ```

- Toate `child_element` care conțin și `subelement1` și `subelement2`:
  ```xpath
  /root_element/child_element[subelement1 and subelement2]
  ```
  Returnează toate cele 3 noduri `child_element`:
  ```xml
  <child_element attribute="example-1" id="c1" status="active">...</child_element>
  <child_element attribute="example-2" id="c2" status="draft">...</child_element>
  <child_element attribute="example-3" id="c3" status="inactive">...</child_element>
  ```

In [15]:
from pathlib import Path
import xml.etree.ElementTree as ET

DTD_TEXT = """\
<!ELEMENT bookstore (book*)>
<!ELEMENT book (title, author, year, price)>
<!ATTLIST book category CDATA #REQUIRED>
<!ELEMENT title (#PCDATA)>
<!ELEMENT author (#PCDATA)>
<!ELEMENT year (#PCDATA)>
<!ELEMENT price (#PCDATA)>
"""

DTD_PATH = Path("bookstore.dtd")
XML_PATH = Path("books.xml")

def write_dtd(dtd_path, dtd_text):
    dtd_path.write_text(dtd_text, encoding="utf-8")
    print(f"[WRITE] DTD -> {dtd_path.resolve()}")

def generate_xml(xml_path):
    root = ET.Element("bookstore")

    def add_book(category, title, author, year, price):
        book = ET.SubElement(root, "book", category=category)
        ET.SubElement(book, "title").text = title
        ET.SubElement(book, "author").text = author
        ET.SubElement(book, "year").text = str(year)
        ET.SubElement(book, "price").text = str(price)

    add_book("fiction", "The Great Gatsby", "F. Scott Fitzgerald", 1925, 10.99)
    add_book("non-fiction", "Sapiens", "Yuval Noah Harari", 2011, 14.99)
    add_book("fiction", "Dune", "Frank Herbert", 1965, 9.99)

    tree = ET.ElementTree(root)
    ET.indent(tree, space="  ", level=0) 
    tree.write(str(xml_path), encoding="utf-8", xml_declaration=True)

    print(f"[WRITE] XML -> {xml_path.resolve()}\n")

def print_dtd_and_xml(dtd_path, xml_path):
    print(dtd_path.read_text(encoding="utf-8"))
    print(xml_path.read_text(encoding="utf-8"))

def process_bookstore(xml_path):
    tree = ET.parse(str(xml_path))
    ET.indent(tree, space="  ", level=0) 
    root = tree.getroot()
    print(f"[INFO] Root tag: {root.tag}")

    for book in root.findall("book"):
        book.set("updated", "false")

        price = book.find("price")
        if price is not None and book.get("category") == "fiction":
            try:
                book.set("updated", "true")
                old_price = float(price.text)
                new_price = round(old_price * 0.9, 2)
                price.text = f"{new_price:.2f}"
            except (TypeError, ValueError):
                pass

    tree.write(str(xml_path), encoding="utf-8", xml_declaration=True)
    print(f"[WRITE] XML updated -> {xml_path.resolve()}\n")

    ET.dump(root)

def xpath_with_etree(file_path):
    tree = ET.parse(str(file_path))
    root = tree.getroot()

    cheap_fiction = []
    modern_authors = []
    categories = []

    for book in root.findall("./book"):
        category = book.get("category")
        if category:
            categories.append(category)

    for book in root.findall("./book[@category='fiction']"):
        price_el = book.find("./price")
        title_el = book.find("./title")
        if price_el is not None and title_el is not None:
            try:
                if float(price_el.text) < 11:
                    cheap_fiction.append(title_el.text)
            except (TypeError, ValueError):
                pass

    for book in root.findall("./book[year]"):
        year_el = book.find("./year")
        author_el = book.find("./author")
        if year_el is not None and author_el is not None:
            try:
                if int(year_el.text) >= 2000:
                    modern_authors.append(author_el.text)
            except (TypeError, ValueError):
                pass

    print("[Query] Cheap fiction:", cheap_fiction)
    print("[Query] Modern authors:", modern_authors)
    print("[Query] Categories:", categories)


write_dtd(DTD_PATH, DTD_TEXT)
generate_xml(XML_PATH)
print_dtd_and_xml(DTD_PATH, XML_PATH)

process_bookstore(XML_PATH)
xpath_with_etree(XML_PATH)

[WRITE] DTD -> C:\Endava\EndevLocal\pcpp\bookstore.dtd
[WRITE] XML -> C:\Endava\EndevLocal\pcpp\books.xml

<!ELEMENT bookstore (book*)>
<!ELEMENT book (title, author, year, price)>
<!ATTLIST book category CDATA #REQUIRED>
<!ELEMENT title (#PCDATA)>
<!ELEMENT author (#PCDATA)>
<!ELEMENT year (#PCDATA)>
<!ELEMENT price (#PCDATA)>

<?xml version='1.0' encoding='utf-8'?>
<bookstore>
  <book category="fiction">
    <title>The Great Gatsby</title>
    <author>F. Scott Fitzgerald</author>
    <year>1925</year>
    <price>10.99</price>
  </book>
  <book category="non-fiction">
    <title>Sapiens</title>
    <author>Yuval Noah Harari</author>
    <year>2011</year>
    <price>14.99</price>
  </book>
  <book category="fiction">
    <title>Dune</title>
    <author>Frank Herbert</author>
    <year>1965</year>
    <price>9.99</price>
  </book>
</bookstore>
[INFO] Root tag: bookstore
[WRITE] XML updated -> C:\Endava\EndevLocal\pcpp\books.xml

<bookstore>
  <book category="fiction" updated="true">
   

## CSV

**`csv`** este un modul standard în Python utilizat pentru lucrul cu fișiere de tip **CSV (Comma-Separated Values)**. Formatul CSV reprezintă un mod simplu de a stoca date tabelare (similar cu un fișier Excel), unde fiecare linie reprezintă un rând, iar valorile sunt separate printr-un delimitator, de obicei **virgulă (`,`)**. Acest format este frecvent folosit pentru importul și exportul datelor între aplicații precum baze de date, foi de calcul și instrumente de analiză.

Funcțiile principale ale modulului `csv` sunt:
- `with open(filename, mode='r', newline='') as file:` — Deschide un fișier CSV.  
    - `filename` — numele fișierului CSV.  
    - `mode` — definește modul de acces:
        - `'r'` — **read**: deschide fișierul pentru citire, utilizat cu `csv.reader()` sau `csv.DictReader()`.  
        - `'w'` — **write**: deschide fișierul pentru scriere (suprascrie conținutul existent), folosit cu `csv.writer()` sau `csv.DictWriter()`.  
        - `'a'` — **append**: adaugă rânduri noi la sfârșitul fișierului.  
        - `'x'` — **exclusive creation**: creează un fișier nou și aruncă eroare dacă acesta există deja.  
    - `newline` — previne inserarea de linii goale suplimentare în fișierele CSV (important pe Windows).  

- `csv.DictWriter(file, fieldnames, *, restval='', extrasaction='raise', dialect='excel', **fmtparams)` — Creează un obiect care scrie rânduri în fișier sub formă de **dicționare**.  
    - `file` — fișier deschis pentru scriere.  
    - `fieldnames` — lista de chei (coloane) care definesc ordinea datelor din fiecare rând.  
    - `restval` — valoare implicită pentru cheile lipsă în dicționarul transmis.  
    - `extrasaction` — stabilește comportamentul pentru cheile suplimentare:  
        - `'raise'` — aruncă eroare (implicit).  
        - `'ignore'` — ignoră cheile necunoscute.  
    - `dialect` — specifică formatul CSV (ex: `excel`, `unix`).  
    - `**fmtparams` — parametri opționali pentru configurarea separatorului, delimitatorului etc.  
    - returnează un obiect writer ce poate scrie date în format CSV pe bază de dicționar.  

- `csv.DictReader(file, fieldnames=None, restkey=None, restval=None, dialect='excel', **fmtparams)` — Creează un obiect care citește fișierul CSV și returnează fiecare rând ca **dicționar**.  
    - `file` — fișier deschis pentru citire.  
    - `fieldnames` — lista de chei (coloane). Dacă este `None`, se preiau automat din prima linie.  
    - `restkey` — numele sub care se stochează valorile în exces față de `fieldnames`.  
    - `restval` — valoare implicită pentru cheile lipsă.  
    - `dialect` — specifică formatul CSV.  
    - `**fmtparams` — parametri suplimentari pentru personalizarea formatului.  
    - returnează un obiect iterabil care livrează fiecare rând ca dicționar.  

- `csv.writer(file, dialect='excel', **fmtparams)` — Creează un obiect care scrie rânduri în fișier ca **liste de valori**.  
    - `file` — fișier deschis pentru scriere.
    - `dialect` — specifică stilul CSV (ex: `excel`, `unix`).  
    - `**fmtparams` — permite configurarea caracterelor de separare, citare, escape etc.  
    - returnează un obiect writer care poate scrie rânduri de date sub formă de liste.  

- `csv.reader(file, dialect='excel', **fmtparams)` — Creează un obiect care citește fișierul CSV rând cu rând, returnând fiecare rând ca listă de valori.  
    - `file` — fișier deschis pentru citire.  
    - `dialect` — specifică formatul CSV.  
    - `**fmtparams` — parametri opționali pentru configurarea delimitatorului (`delimiter`), caracterului de citare (`quotechar`), escape, etc.  
    - returnează un obiect iterabil care livrează fiecare rând sub formă de listă.  

- `writer.writeheader()` — Scrie linia de antet (headerele) în fișier, pe baza listei de `fieldnames` definită la inițializarea `DictWriter`.  
    - returnează numărul de caractere scrise.  

- `writer.writerow(row)` — Scrie un singur rând în fișierul CSV.  
    - `row` — poate fi fie:  
        - un **dicționar**, dacă este folosit cu `DictWriter`, caz în care cheile corespund valorilor din `fieldnames`;  
        - o **listă** sau un **tuplu**, dacă este folosit cu `writer`, caz în care ordinea elementelor definește coloanele.  
    - returnează numărul de caractere scrise în fișier.  

In [4]:
import csv
from pathlib import Path

CSV_FILE = Path("items.csv")

fieldnames = ["name", "quantity"]
with open(CSV_FILE, "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({"name": "laptop", "quantity": 3})
    writer.writerow({"name": "mobile phone", "quantity": 2})
    writer.writerow({"quantity": 1, "name": "car"})
print(f"[WRITE DictWriter] File created at: {CSV_FILE.resolve()}")

with open(CSV_FILE, "r", newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    print("[READ DictReader]", reader.fieldnames)
    for row in reader:
        print("[READ DictReader]", row)
    print()

with open(CSV_FILE, "a", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["pen", 10])
    writer.writerow(["notebook", 5])
print("[APPEND writer] Added 2 new rows.")

with open(CSV_FILE, "r", newline="") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        print("[READ reader]", line)

[WRITE DictWriter] File created at: C:\Endava\EndevLocal\pcpp\items.csv
[READ DictReader] ['name', 'quantity']
[READ DictReader] {'name': 'laptop', 'quantity': '3'}
[READ DictReader] {'name': 'mobile phone', 'quantity': '2'}
[READ DictReader] {'name': 'car', 'quantity': '1'}

[APPEND writer] Added 2 new rows.
[READ reader] ['name', 'quantity']
[READ reader] ['laptop', '3']
[READ reader] ['mobile phone', '2']
[READ reader] ['car', '1']
[READ reader] ['pen', '10']
[READ reader] ['notebook', '5']


## INI

Un **fișier `.ini`** este un format text simplu, folosit pentru **stocarea configurațiilor aplicațiilor**. Este structurat în **secțiuni**, fiecare conținând **opțiuni de tip cheie--valoare**. Sintaxa este clară și ușor de citit, motiv pentru care acest format este frecvent utilizat pentru fișiere de configurare. Fișierele pot conține o secțiune specială default, care definește valori implicite accesibile tuturor secțiunilor, dacă acestea nu sunt suprascrise. Structura unui fișier INI este următoarea:
``` ini
[DEFAULT]
default_key = default_value
default_key2 = default_value2

[section]
key = value
key2 = value2
```

`configparser` este un modul standard Python folosit pentru a citi, scrie și manipula fișiere INI. Oferă o interfață convenabilă pentru lucrul cu configurații în format text.

Funcțiile și metodele principale ale modulului sunt:
- `with open(filename, mode='r', encoding='utf-8') as file:` — Deschide fișierul INI.
    - `filename` — numele fișierului de configurare.
    - `mode` — modul de acces:
        - `'r'` — **read**: deschide fișierul pentru citire, folosit împreună cu `config.read_file(file)` pentru a încărca datele din fișierul INI.
        - `'w'` — **write**: deschide fișierul pentru scriere (suprascrie conținutul existent), folosit cu `config.write(file)` pentru a salva modificările.
        - `'a'` — **append**: deschide fișierul pentru adăugare la final, fără a șterge datele existente.
        - `'x'` — **exclusive creation**: creează un fișier nou și ridică `FileExistsError` dacă există deja.
    - `encoding` — setul de caractere folosit la citire/scriere (implicit 'utf-8').

- `configparser.ConfigParser(defaults=None, *, dict_type=dict, allow_no_value=False, delimiters=('=', ':'), comment_prefixes=('#', ';'))` - Creează un obiect pentru gestionarea fișierelor de configurare.
    - `defaults` - dicționar cu valori implicite (corespunde secțiunii \[DEFAULT\]).
    - `dict_type` - tipul de dicționar intern folosit (implicit `dict`).
    - `allow_no_value` - permite existența opțiunilor fără valoare (cheie=).
    - `delimiters` - caracterele care separă cheia de valoare (= sau :).
    - `comment_prefixes` - caractere folosite pentru comentarii (# și ;).
    - returnează un obiect de tip `ConfigParser`.

- `ConfigParser.read(filenames, encoding=None)` - Citește conținutul unui fișier INI de pe disc.
    - `filenames` - numele fișierului (sau o listă de fișiere) de citit.
    - `encoding` - setul de caractere folosit pentru citire (utf-8 implicit).
    - returnează o listă cu fișierele citite cu succes.

- `ConfigParser.read_string(string)` - Încarcă o configurație direct dintr-un șir de caractere.
    - `string` - conținutul complet al fișierului INI, sub formă de text.

- `ConfigParser.read_dict(dictionary)` - Încarcă o configurație dintr-un dicționar Python.
    - `dictionary` - un dicționar care mapează numele secțiunilor la alte dicționare.

- `ConfigParser.add_section(section)` - Creează o secțiune nouă în configurație.
    - `section` - numele noii secțiuni.
    - Aruncă `DuplicateSectionError` dacă secțiunea există deja.

- `ConfigParser.set(section, option, value)` - Setează o valoare pentru o cheie în secțiunea specificată. Dacă cheia există se actualizează, dacă nu, se crează.
    - `section` - numele secțiunii.
    - `option` - numele noii opțiunii.
    - `value` - valoarea atribuită opțiunii.
    - echivalent cu accesarea directă: `config[section][option] = value`.

- `ConfigParser.defaults()` - Obține dicționarul cu valorile implicite definite în `[DEFAULT]`.
    - returnează un dicționar cu opțiunile și valorile implicite.

- `ConfigParser.items(section=_UNSET)` - Returnează toate opțiunile dintr-o secțiune.
    - `section` - numele secțiunii.
    - returnează o listă de tuple `(cheie, valoare)`.

- `ConfigParser.remove_option(section, option)` - Elimină o cheie dintr-o secțiune.
    - `section` - numele secțiunii.
    - `option` - numele opțiunii.
    - returnează `True` dacă opțiunea a fost ștearsă, `False` dacă nu a existat.

- `ConfigParser.getint(section, option, *, fallback=None)` - Citește o valoare și o convertește la `int`.
    - `section` - numele secțiunii.
    - `option` - numele opțiunii.
    - `fallback` - valoarea returnată dacă opțiunea cerută nu există 
    - returnează un număr întreg (`int`).

- `ConfigParser.getfloat(section, option, *, fallback=None)` - Citește o valoare și o convertește la `float`.
    - `section` - numele secțiunii.
    - `option` - numele opțiunii.
    - `fallback` - valoarea returnată dacă opțiunea cerută nu există 
    - returnează un număr zecimal (`float`).

- `ConfigParser.getboolean(section, option, *, fallback=None)` - Citește o valoare și o convertește la `bool`.
    - `section` - numele secțiunii.
    - `option` - numele opțiunii.
    - `fallback` - valoarea returnată dacă opțiunea cerută nu există 
    - acceptă variante precum `yes/no`, `true/false`, `on/off`, `1/0`.
    - returnează `True` sau `False`.

- `ConfigParser.has_section(section)` - Verifică dacă o secțiune există.
    - `section` - numele secțiunii.
    - returnează `True` dacă secțiunea există, altfel `False`.

- `ConfigParser.has_option(section, option)` - Verifică dacă o cheie există într-o secțiune.
    - `section` - numele secțiunii.
    - `option` - numele opțiunii.
    - returnează `True` dacă opțiunea există, altfel `False`.

- `ConfigParser.sections()` - Returnează o listă cu toate secțiunile definite (fără `[DEFAULT]`).
    - returnează o listă de șiruri (numele secțiunilor).

- `ConfigParser.write(file_object, space_around_delimiters=True)` - Scrie configurația înapoi într-un fișier INI.
    - `file_object` - fișier deschis pentru scriere.
    - `space_around_delimiters` - dacă se lasă spațiu între cheie, delimitator și valoare (`key = value`).

In [5]:
import configparser
from pathlib import Path

config_file = Path("settings.ini")
default_text = """
[DEFAULT]
company = TechCorp
location = EU
"""
config_file.write_text(default_text)
print(f"[WRITE] Created {config_file.resolve()} with default content.\n")

config = configparser.ConfigParser()
read_files = config.read(config_file)
print("[READ] Files successfully loaded:", read_files, '\n')

config.read_string("""
[database]
host = localhost
port = 3306
timeout = 3.5
secure = yes
""")

config.read_dict({
    "api": {"endpoint": "https://api.example.com", "retries": "5", "active": "true"}
})

config.add_section("app")
config.set("app", "debug", "True")
config.set("app", "language", "Python")
config["app"]["version"] = "1.0.0"

print("[app after creation]")
for k, v in config["app"].items():
    if k not in config.defaults():
        print(f"{k} = {v}")
print()

config.set("app", "debug", "False")
config["app"]["language"] = "C++"
config.remove_option("app", "version")

print("[app after modification]")
for k, v in config["app"].items():
    if k not in config.defaults():
        print(f"{k} = {v}")
print()

port = config.getint("database", "port")
timeout = config.getfloat("database", "timeout")
secure = config.getboolean("database", "secure")
debug = config.getboolean("app", "debug")

print("[getint port]:", port)
print("[getfloat timeout]:", timeout)
print("[getboolean secure]:", secure)
print("[getboolean debug]:", debug)
print("[has_section 'user']:", config.has_section("app"))
print("[has_option 'database','host']:", config.has_option("database", "host"), '\n')

if config.defaults():
    print("[DEFAULT]")
    for k, v in config.defaults().items():
        print(f"{k} = {v}")
    print()

for section in config.sections():
    print(f"[{section}]")
    for k in config[section]:
        if k not in config.defaults():
            print(f"{k} = {config[section][k]}")
    print()

with open(config_file, "w") as f:
    config.write(f)
print(f"[WRITE] Combined configuration saved to: {config_file.resolve()}")

[WRITE] Created C:\Endava\EndevLocal\pcpp\settings.ini with default content.

[READ] Files successfully loaded: ['settings.ini'] 

[app after creation]
debug = True
language = Python
version = 1.0.0

[app after modification]
debug = False
language = C++

[getint port]: 3306
[getfloat timeout]: 3.5
[getboolean secure]: True
[getboolean debug]: False
[has_section 'user']: True
[has_option 'database','host']: True 

[DEFAULT]
company = TechCorp
location = EU

[database]
host = localhost
port = 3306
timeout = 3.5
secure = yes

[api]
endpoint = https://api.example.com
retries = 5
active = true

[app]
debug = False
language = C++

[WRITE] Combined configuration saved to: C:\Endava\EndevLocal\pcpp\settings.ini


## SQLite3

**`sqlite3`** este un modul standard în Python care oferă o interfață directă către motorul de baze de date **SQLite** — o bază de date relațională ușoară, fără server, care stochează datele într-un fișier local. Este ideală pentru aplicații mici și medii, testare rapidă și prototipuri, fără a necesita configurarea unui server de baze de date extern. SQLite suportă **standardul SQL (Structured Query Language)** și este complet integrat în Python, permițând lucrul cu tabele, interogări și tranzacții într-un mod simplu și portabil.

Fluxul general de lucru cu `sqlite3` este:
1. Se stabilește o conexiune la o bază de date (`connect()`).

2. Se creează un obiect cursor (`cursor()`).

3. Se execută comenzi SQL (`execute()` / `executemany()`).

4. Se preiau rezultatele (`fetchone()`, `fetchall()`).

5. Se confirmă modificările (`commit()`).

6. Se închid cursorul și conexiunea (`close()`).

Funcțiile principale ale modulului `sqlite3` sunt:
- `sqlite3.connect(database, timeout=5.0, detect_types=0, isolation_level=None, check_same_thread=True, factory=sqlite3.Connection, uri=False)` — Creează o conexiune la o bază de date SQLite.  
    - `database` — numele fișierului bazei de date (ex: `"data.db"`). Dacă fișierul nu există, va fi creat automat. Poate fi folosit și `":memory:"` pentru a crea o **bază de date temporară în memorie**, care dispare la închiderea programului.  
    - `timeout` — timpul de așteptare (în secunde) pentru accesarea bazei de date dacă este blocată.  
    - `detect_types` — controlează detectarea automată a tipurilor de date.  
    - `check_same_thread` — dacă este `True`, conexiunea nu poate fi folosită din alte thread-uri.  
    - `uri` — dacă este `True`, permite folosirea unei adrese de tip URI.  
    - returnează un obiect de tip `Connection`.  

- `conn.cursor()` — Creează un **cursor**, folosit pentru a executa comenzi SQL asupra bazei de date.  
    - returnează un obiect `Cursor` care permite interacțiunea cu baza de date.  

- `cursor.execute(sql, parameters=None)` — Execută o instrucțiune SQL unică.  
    - `sql` — comanda SQL de executat (ex: `SELECT`, `INSERT`, `UPDATE`, `DELETE`).  
    - `parameters` — opțional, un tuplu cu valorile care vor fi inserate în comanda SQL (folosind `?` ca substituent).  
    - returnează obiectul cursor, pentru a permite apeluri succesive precum `fetchone()` sau `fetchall()`.  

- `cursor.executemany(sql, seq_of_parameters)` — Execută aceeași instrucțiune SQL pentru mai multe seturi de date.  
    - `sql` — comanda SQL de executat (de obicei `INSERT`).  
    - `seq_of_parameters` — o listă de tupluri, fiecare reprezentând un set de valori pentru o execuție.  
    - returnează obiectul cursor, după executarea multiplelor inserări sau actualizări.  

- `cursor.fetchone()` — Returnează **primul rând** din rezultatul ultimei interogări.  
    - returnează un tuplu care conține valorile unui singur rând, sau `None` dacă nu există rezultate.  

- `cursor.fetchall()` — Returnează **toate rândurile** rezultate în urma unei interogări.  
    - returnează o listă de tupluri, unde fiecare tuplu reprezintă un rând din rezultatul interogării.  

- `conn.commit()` — Salvează permanent modificările efectuate asupra bazei de date. Este necesar pentru operațiile de modificare (INSERT, UPDATE, DELETE) înainte de închiderea conexiunii.  

- `cursor.close()` — Închide cursorul curent. Eliberează resursele asociate cursorului, dar conexiunea rămâne activă.  

- `conn.close()` — Închide conexiunea cu baza de date. Este important de apelat pentru a elibera resursele și a asigura salvarea datelor.  

SQLite este **dinamic tipizat**, ceea ce înseamnă că nu impune tipurile la nivel de coloană strict, ci la nivel de valoare. Tipurile de date suportate în SQLite sunt:
| SQLite      | Python           |
|-----|------|
| `NULL`      | `None`           |
| `INTEGER`   | `int`            |
| `REAL`      | `float`          |
| `TEXT`      | `str`            |
| `BLOB`      | `bytes`          |


Comenzile SQL de bază sunt:
- **CREATE** - creare tabel:
  ```sql
  CREATE TABLE users (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      name TEXT NOT NULL,
      age INTEGER,
      country TEXT
  );
  ```

- **READ** - citire date:
  ```sql
  SELECT * FROM users WHERE country = 'USA';
  ```

- **UPDATE** - actualizare date:
  ```sql
  UPDATE users SET age = 31 WHERE name = 'Alice';
  ```

- **DELETE** - ștergere date:
  ```sql
  DELETE FROM users WHERE id = 3;
  ```

- **INSERT** - inserare date:
  ```sql
  INSERT INTO users (name, age, country) VALUES ('George', 30, 'Romania');
  ```

In [12]:
import sqlite3
from pathlib import Path

DB_PATH = Path("db.sqlite3")

conn = sqlite3.connect(DB_PATH)
print("[CONNECT] database at:", DB_PATH.resolve())

try:
    cur = conn.cursor()
    print("[CURSOR] cursor created")

    cur.execute("DROP TABLE IF EXISTS users")
    cur.execute("""
        CREATE TABLE users (
            id      INTEGER PRIMARY KEY AUTOINCREMENT,
            name    TEXT NOT NULL,
            age     INTEGER,
            country TEXT
        )
    """)
    print("[EXECUTE] table created")

    users = [
        ("Alice", 30, "USA"),
        ("Bob",   25, "UK"),
        ("Carla", 28, "RO"),
    ]
    cur.executemany(
        "INSERT INTO users(name, age, country) VALUES (?, ?, ?)",
        users
    )
    print("[EXECUTEMANY] inserted", len(users), "rows")

    cur.execute(
        "INSERT INTO users(name, age, country) VALUES (?, ?, ?)",
        ("Dumitru", 33, "MD")
    )
    print("[EXECUTE] inserted 1 row")

    conn.commit()
    print("[COMMIT] changes saved")

    cur.execute("SELECT id, name, age, country FROM users WHERE name = ?", ("Alice",))
    one = cur.fetchone()
    print("[FETCHONE]", one)

    cur.execute("SELECT * FROM users ORDER BY id")
    all_rows = cur.fetchall()
    print("[FETCHALL] total rows:", len(all_rows))
    for r in all_rows:
        print("  ", r)
finally:
    try:
        cur.close()
        print("[CLOSE] cursor closed")
    except Exception:
        pass
    conn.close()
    print("[CLOSE] connection closed")

[CONNECT] database at: C:\Endava\EndevLocal\pcpp\db.sqlite3
[CURSOR] cursor created
[EXECUTE] table created
[EXECUTEMANY] inserted 3 rows
[EXECUTE] inserted 1 row
[COMMIT] changes saved
[FETCHONE] (1, 'Alice', 30, 'USA')
[FETCHALL] total rows: 4
   (1, 'Alice', 30, 'USA')
   (2, 'Bob', 25, 'UK')
   (3, 'Carla', 28, 'RO')
   (4, 'Dumitru', 33, 'MD')
[CLOSE] cursor closed
[CLOSE] connection closed
